In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw4

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

In [ ]:
import torch
from transformers import GPT2Tokenizer, AutoTokenizer, GPT2LMHeadModel
from datasets import load_dataset
from collections import defaultdict

## **Task 1: Byte pair encoding**

The goal is to calculate byte pair encodings for a given dataset. For a recap of the concept we refer to lecture 9, slides 22.ff and internet search.

### Task 1.1
First we want to make sure we create a suffieciently large corpus. As dataset you import the dataset "wikitext-2-raw-v1" from huggingface. Make sure to use the 500 first entries of the training dataset and subsequently filter out empty  entries. Also print the first entry of the corpus.

In [ ]:
# Load WikiText-2 dataset "wikitext-2-raw-v1"
corpus = []
# YOUR CODE HERE

Note that the corpus should have a format like
corpus = [
    "This is the first sentence",
    "This is the second sentence",
    ..
]. For debugging during coding you can also consider working with a smaller corpus, e.g. only 5 sentences

### Task 1.2
Here, you loop through the corpus and count the word frequencies

In [ ]:
# Initialize a tokenizer from the transformers library
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Initialize a defaultdict to store word frequencies
word_freqs = defaultdict(int)
# this dictionary gets entries for word_freqs[word]

alphabet = []
vocab = [""]
splits = {}
vocab_size = 200
merges = {}

# We loop through to corups to calculate word frequencies
for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    # YOUR CODE HERE to count word frequencies



The following block creates the alphabet and initial vocabulary

In [ ]:
for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print("alphabet",alphabet)
vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}

### Task 1.3
Define a function to compute the frequency of each pair of characters

In [ ]:
# Define a function to compute the frequency of each pair of characters
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    # YOUR CODE HERE to compute the frequency of each pair of characters

    return pair_freqs

# Initialize pair_freqs by calling the compute_pair_freqs function
pair_freqs = compute_pair_freqs(splits)

Here we find the pair of characters that appears most frequently together

In [ ]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

### Task 1.4
Define a function to merge the most frequent pair of characters

In [ ]:

# Define a function to merge the most frequent pair of characters
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        # YOUR CODE HERE to merge the most frequent pair of characters in each word

    return splits


### Task 1.5

Iterate this process until you reach a predefined vocabulary size

In [ ]:
# YOUR CODE HERE to iterate this process until you reach a predefined vocabulary size



We provide the tokenize function to you

In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])
tokenize("Darmstadt holds the official title City of Science (German: Wissenschaftsstadt) as it is a major centre of scientific institutions, universities, and high-technology companies.")

### Task 1.6
Now, repeat the byte pair encoding for vocab sizes of 1000 and 5000. What do you notice?

In [ ]:
# Use this chunk for the answer

In [ ]:
# Use this chunk for the answer

# **Task 2: Getting to Know Foundation Models**

In this task, we want to load two different versions of GPT-2 and compare their performance.

### Task 2.1

After we imported transformers from Huggingface, your first task is to initialize the GPT2LMHead Model. This model should be a gpt2 model from huggingface under the variable model. Also define the GPT2 tokenizer.

While we have seen the transformers in the lectures, you should print a single GPT2Block to the console for the 10th layer. Then, count the parameters of the model.

In [ ]:
# Print a GPT2Block in the 10th layer

def count_parameters(model):
    #YOUR CODE HERE

# GPT2_small parameter trainable parameters:
print(f"The model has {count_parameters(model_small):,} trainable parameters")

### Task 2.2
Because we want to compare two versions of GPT, you now also load the gpt2-large from huggingface as model. As evaluation dataset you import the dataset "wikitext-2-raw-v1" from huggingface. (like in Task1) Make sure to use the 200 first entries of the test dataset and print an entry. Also print the parameters of the gpt2-large model

In [ ]:
# Load the gpt2-large model from huggingface

# Load the dataset

# Print any single entry

# Print Trainable parameters of huggingface gpt2-large

### Task 2.3
As a next step, we define an evaluation metric to calculate predictions. Remember the perplexity funciton from the lecture. The function should take a model, tokenizer and a text and return the perplexity score for the given text.

Hint: you should encode the text, generate outputs for inputs and corresponding labels and then calculate perplexity.

In [ ]:
def calculate_perplexity(model, tokenizer, text):
    # YOUR CODE HERE

In [ ]:
perplexities_small = []
perplexities_large = []
# Loop over the dataset, append scores to the lists

# Calculate and print average perplexities
print("Average perplexity (GPT2 ): ", np.nanmean(perplexities_small))
print("Average perplexity (GPT2 large): ", np.nanmean(perplexities_large))

What do you notice? Please explain.

### Task 2.4 Masking attention
What is an attention mask? When and why is it usually used?

Your answer here:

Your task is now to run a prompt in regular inference, but with an attention mask. You can reuse the gpt2_small model

In [ ]:
text_prompt = "The sun was setting behind"

# Step 1: Implement attention masking
def apply_attention_mask(input_ids, mask_start_idx, mask_end_idx):
    # -----------------------
    #YOUR CODE HERE
    # -----------------------

    #return attention_mask

# Step 2: Apply attention masks
mask_start = 3  # Start index of the region to mask
mask_end = 5  # End index of the region to mask
input_ids = tokenizer_small.encode(text_prompt, return_tensors='pt')
attention_mask = apply_attention_mask(input_ids, mask_start, mask_end)

# Step 3: Generate output using modified attention mechanism
with torch.no_grad():
    outputs = model_small.generate(input_ids, attention_mask=attention_mask)
    generated_text = tokenizer_small.decode(outputs[0], skip_special_tokens=True)

# Step 4: Compare and analyze the generated outputs
print("Text Prompt:", text_prompt)
print("Generated Text:", generated_text)



In [ ]:
# Step 5: Run the same prompt without attention mask

# YOUR CODE HERE

Step 6: Compare the two outputs and describe it in one to two sentences

Your answer here: